In [1]:
import pandas as pd
from pathlib import Path
import win32com.client as win32

# Leitura dos arquivos:
email_df = pd.read_excel(r'Base de Dados\Emails.xlsx')
lojas_df = pd.read_csv(r'Base de Dados\Lojas.csv', sep=';', encoding='latin-1')
vendas_df = pd.read_excel(r'Base de Dados\Vendas.xlsx')

completo_df = vendas_df.merge(lojas_df, on='ID Loja')

display(completo_df)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,21,2019-01-02,1,Camisa Gola V Listrado,2,116,232,Iguatemi Esplanada
4,34,2019-01-02,1,Sapato Listrado,1,363,363,Iguatemi Esplanada
...,...,...,...,...,...,...,...,...
100994,69846,2019-12-25,8,Short Estampa,2,96,192,Salvador Shopping
100995,69846,2019-12-25,8,Tênis Estampa,5,256,1280,Salvador Shopping
100996,69850,2019-12-25,8,Calça Estampa,4,177,708,Salvador Shopping
100997,69972,2019-12-26,8,Terno Liso,3,720,2160,Salvador Shopping


In [2]:
#Verificando os dados:
completo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100999 entries, 0 to 100998
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Código Venda    100999 non-null  int64         
 1   Data            100999 non-null  datetime64[ns]
 2   ID Loja         100999 non-null  int64         
 3   Produto         100999 non-null  object        
 4   Quantidade      100999 non-null  int64         
 5   Valor Unitário  100999 non-null  int64         
 6   Valor Final     100999 non-null  int64         
 7   Loja            100999 non-null  object        
dtypes: datetime64[ns](1), int64(5), object(2)
memory usage: 6.9+ MB


In [3]:
#Separando o dataframe de cada loja:
dic_lojas = {}

for loja in lojas_df['Loja']:
    dic_lojas[loja] = completo_df[completo_df['Loja'] == loja]

In [4]:
#Pegando o último dia da planilha:
dia_indicador = completo_df['Data'].max()

26/12/2019


In [5]:
#Salvando os dataframes de cada loja em sua pasta de Backup:
caminho = Path('Backup Arquivos Lojas')

for loja in dic_lojas:
    pasta_loja = caminho / f'{loja}'
    pasta_loja.mkdir(exist_ok=True)

    nome_arquivo = f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_{loja}.xlsx'
    local_arquivo = pasta_loja / nome_arquivo
    dic_lojas[loja].to_excel(local_arquivo)

In [6]:
#Metas:
META_FATURAMENTO_DIA = 1000
META_FATURAMENTO_ANO = 1650000
META_QTDE_PRODUTOS_DIA = 4
META_QTDE_PRODUTOS_ANO = 120
META_TICKETMEDIO_DIA = 500
META_TICKETMEDIO_ANO= 500

In [12]:
def verificar_cor(indicativo, meta):
    if indicativo >= meta:
        return 'green'
    else:
        return 'red'

for loja in dic_lojas:

    vendas_loja = dic_lojas[loja]
    vendas_loja_dia = vendas_loja[vendas_loja['Data']==dia_indicador]
    
    #Faturamento:
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    #Diversidade de Produtos:
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
    
    #Ticket Médio:
    valor_venda = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()


nan
